In [2]:
#Import libraries
import requests
import pandas as pd
from datetime import datetime
import os
import html

In [3]:
#Check what to do with validations-reseau.csv?
#Need PCA bc units are different for variabless

In [4]:
#Download dataset
df_lines = pd.read_csv("./Online_Data/referentiel-des-lignes.csv", sep=';')
stops_data = pd.read_csv("./Online_Data/arrets.csv", sep=';')

trafic2023_ratio = pd.read_csv("./Online_Data/validations-1er-semestre.csv", sep=';')
trafic2023_raw = pd.read_csv("./Online_Data/validations-reseau.csv", sep=";")

hourly_weather = pd.read_csv("./Online_Data/Weather/hourly_weather.csv")
minutely_15_weather = pd.read_csv("./Online_Data/Weather/minutely_15_weather.csv")

metro_incident = pd.read_csv("./Collected_Data/metro_line_reports.csv")
rer_incident = pd.read_csv("./Collected_Data/rer_line_reports.csv")

df_holidays = pd.read_csv("./Online_Data/vacances-scolaires-par-zone.csv", sep = ";")
df_bank_holidays = pd.read_csv("./Online_Data/jours_feries_metropole.csv", sep=",")

In [5]:
delays_metro = pd.read_csv("./Collected_Data/metro_delays.csv")
delays_rer = pd.read_csv("./Collected_Data/rail_delays.csv")
onTime_metro = pd.read_csv("./Collected_Data/metro_onTime.csv", low_memory=False)
onTime_rer = pd.read_csv("./Collected_Data/rail_onTime.csv", low_memory=False)

In [6]:
delays_metro2 = pd.read_csv("./Collected_Data/metro_delays2.csv")
delays_rer2 = pd.read_csv("./Collected_Data/rail_delays2.csv")
onTime_metro2 = pd.read_csv("./Collected_Data/metro_onTime2.csv", low_memory=False)
onTime_rer2 = pd.read_csv("./Collected_Data/rail_onTime2.csv", low_memory=False)

In [7]:
delays_metro = pd.concat([delays_metro, delays_metro2], ignore_index=True)
delays_rer = pd.concat([delays_rer, delays_rer2], ignore_index=True)
onTime_metro = pd.concat([onTime_metro, onTime_metro2], ignore_index=True)
onTime_rer = pd.concat([onTime_rer, onTime_rer2], ignore_index=True)

In [8]:
#Create filter
stops_filter = [22086, 463013, 22136, 462993, 21964, 462969, 22125, 463113, 41295, 473921, 473993, 41354, 474060, 474061]
name_filter = ["CH.D.G.ETOILE", "CHATELET", "SAINT-LAZARE","ST-GERM.D.PRES", "BLANCHE", "AVENUE DU PRESIDENT KENNEDY", "BUNO GIRONVILLE", "MASSY PALAISEAU"]
lines_filter = ["C01371", "C01372", "C01374" ," C01382", "C01742", "C01743", "C01727", "C0172"]

In [9]:
#Clean datasets
line_refs = df_lines[(~df_lines['TransportSubmode'].isin(['suburbanRailway', 'regionalRail', 'railShuttle']))]
line_refs = line_refs[['ID_Line', 'TransportMode', 'Name_Line']]
line_refs = line_refs.sort_values(by='ID_Line')

stops_data = stops_data[stops_data['ArRType'].isin(['metro', 'rail'])]
stops_data = stops_data.sort_values(by=['ArRType', 'ArRId'])
stops_data = stops_data[['ArRId', 'ArRName', 'ArRType', 'ArRTown']]

def cleaning_message(df):
    df['ref'] = df['ref'].str.replace('stop_point:IDFM:', '', regex=False)
    df['ref'] = df['ref'].str.replace('line:IDFM:', '', regex=False)
    df['ref'] = df['ref'].str.replace('stop_area:IDFM:', '', regex=False)
    df['message_text'] = df['message_text'].str.replace('<p>', '', regex=False)
    df['message_text'] = df['message_text'].str.replace('<br>', '', regex=False) 
    df = df[df["channel_name"].isin(["moteur"])]

cleaning_message(metro_incident)
cleaning_message(rer_incident)

metro_incident['message_text'] = metro_incident['message_text'].str.split('</p>').str[0]
metro_incident.drop_duplicates(subset=["disruption_id"], inplace=True)
metro_incident.drop(['tags', 'category', 'updated_at', 'channel_name'], axis=1, inplace=True) 

rer_incident['message_text'] = rer_incident['message_text'].str.replace('</p>', '', regex=False)
rer_incident['message_text'] = rer_incident['message_text'].apply(html.unescape)
rer_incident.drop_duplicates(subset=["disruption_id"], inplace=True)
rer_incident.drop(['tags', 'category', 'updated_at', 'channel_name'], axis=1, inplace=True) 

def refs(df):
    df['line_ref'] = df['line_ref'].str.replace('STIF:Line::', '', regex=False).str.rstrip(':')
    df['stop_reference'] = pd.to_numeric(df['stop_reference'], errors='coerce')
    df['stop_reference'] = df['stop_reference'].fillna(0).astype('int64')

refs(delays_metro)
refs(delays_rer)
refs(onTime_metro)
refs(onTime_rer)

delays_metro.drop(['scheduled_arrival','scheduled_departure','arrival_difference','departure_difference'], axis=1, inplace=True)
onTime_metro.drop(['scheduled_arrival','scheduled_departure','arrival_difference','departure_difference'], axis=1, inplace=True)

mapping = dict(zip(name_filter, stops_filter))
trafic2023_ratio['LIBELLE_ARRET_REA'] = trafic2023_ratio['LIBELLE_ARRET'].replace(mapping) #issue -  incorrect mapping? stop number refers to wrong stop
trafic2023_ratio = trafic2023_ratio[trafic2023_ratio['LIBELLE_ARRET_REA'].isin(stops_filter)]

#trafic2023_raw['LIBELLE_ARRET_REA'] = trafic2023_raw['LIBELLE_ARRET'].replace(mapping) #don't need anymore?
#trafic2023_raw.drop(["lda"],axis=1, inplace=True)

In [10]:
#Holidays and day types
df_bank_holidays["Date"] = pd.to_datetime(df_bank_holidays["date"])
df_holidays["Date"] = pd.to_datetime(df_holidays["Date"])

df_with_holidays = df_holidays.merge(df_bank_holidays, on = "Date", how = "left")
df_with_holidays = df_with_holidays.sort_values(by='Date')

df_with_holidays['holiday_type'] = df_with_holidays['nom_jour_ferie'].combine_first(df_with_holidays['Nom de la période'])
df_with_holidays['day_of_week'] = df_with_holidays['Date'].dt.weekday

def classify_day(row):
    if row['day_of_week'] < 5:  # Weekdays (Monday to Friday)
        if pd.notna(row['holiday_type']):
            return 'JOVS'  # Weekday with a holiday
        else:
            return 'JOHV'  # Weekday without a holiday
    elif row['day_of_week'] == 5:  # Saturday
        if pd.notna(row['holiday_type']):
            return 'SAVS'  # Saturday with a holiday
        else:
            return 'SAHV'  # Saturday without a holiday
    elif row['day_of_week'] == 6:  # Sunday
        return 'DIJFP'  # Sunday (always labeled DIJFP)
    return None

df_with_holidays['day_type'] = df_with_holidays.apply(classify_day, axis=1)
df_with_holidays["is_bank_holiday"] = (df_with_holidays["nom_jour_ferie"]).notna().astype(int)
df_with_holidays["is_holiday"] = (df_with_holidays["Nom de la période"]).notna().astype(int)
df_with_holidays["saturday"]= (df_with_holidays["Date"].dt.weekday == 5).astype(int)
df_with_holidays["sunday"]=  (df_with_holidays["Date"].dt.weekday == 6).astype(int)
df_with_holidays["is_weekend"]= df_with_holidays["Date"].dt.weekday.isin([5,6]).astype(int)
df_with_holidays["is_weekend_or_bank_holiday"] = df_with_holidays[["is_weekend", "is_bank_holiday"]].max(axis=1)

df_with_holidays.drop(['timestamp_unix', 'date', 'annee', 'zone', 'Calendrier Zone A', 'Calendrier Zone B', 'Calendrier Zone C'], axis=1, inplace=True)


In [11]:
#Filter dataset
stops_data = stops_data[stops_data['ArRId'].isin(stops_filter)]

metro_incident = metro_incident[
    metro_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

rer_incident = rer_incident[
    rer_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

trafic2023_ratio[['start_hour', 'end_hour']] = trafic2023_ratio['TRNC_HORR_60'].str.extract(r'(\d+)H-(\d+)H').dropna().astype(int)
trafic2023_ratio.drop(['CODE_STIF_RES', 'CODE_STIF_ARRET', 'LIBELLE_ARRET_REA', 'lda'], axis=1, inplace=True)

trafic2023_ratio_rer = trafic2023_ratio[trafic2023_ratio['CODE_STIF_TRNS'].isin([810, 800])]
trafic2023_ratio_metro = trafic2023_ratio[trafic2023_ratio['CODE_STIF_TRNS'].isin([100])]

In [12]:
#don't need?
#trafic2023_raw = trafic2023_raw[trafic2023_raw['LIBELLE_ARRET_REA'].isin(stops_filter)] 

In [13]:
#Merge datasets
merged_metro = pd.concat([delays_metro, onTime_metro], ignore_index=True)
merged_metro = pd.merge(merged_metro, stops_data, left_on='stop_reference', right_on='ArRId')
merged_metro.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode', 'recorded_at_time'], axis=1, inplace=True) 

merged_rer = pd.concat([delays_rer, onTime_rer], ignore_index=True)
merged_rer = pd.merge(merged_rer, stops_data, left_on='stop_reference', right_on='ArRId')
merged_rer.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode', 'recorded_at_time'],  axis=1, inplace=True)

def date_format(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'].str.replace('Z', ''), errors='coerce')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['real_arrival'] = pd.to_datetime(df['real_arrival']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['real_arrival'] = pd.to_datetime(df['real_arrival'])
    df['real_departure'] = pd.to_datetime(df['real_departure']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['real_departure'] = pd.to_datetime(df['real_departure'])


def format_rer(df):
    df['scheduled_arrival'] = pd.to_datetime(df['scheduled_arrival']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['scheduled_arrival'] = pd.to_datetime(df['scheduled_arrival'])
    df['scheduled_departure'] = pd.to_datetime(df['scheduled_departure']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['scheduled_departure'] = pd.to_datetime(df['scheduled_departure'])
    df.loc[(df['arrival_difference'] >= 3) | (df['departure_difference'] >= 3), 'departure_status'] = 'delayed'
    df['nearest_datetime'] = df['scheduled_arrival'].combine_first(df['scheduled_departure'])

date_format(merged_metro)
date_format(merged_rer)
format_rer(merged_rer)

merged_metro = merged_metro.sort_values(by='real_arrival')
merged_rer = merged_rer.sort_values(by='nearest_datetime')


In [14]:
#Clean Gas Price data
def clean_gas_data(file_path, fuel_type):
    df = pd.read_csv(file_path, skiprows=3, sep=';')
    df.rename(columns={df.columns[1]: f'{fuel_type}'}, inplace=True)
    df = df.iloc[:, :-1]
    return df
    
gas_95_df = clean_gas_data("./Online_Data/Gas/octane_95.csv", "95")
gas_98_df = clean_gas_data("./Online_Data/Gas/octane_98.csv", "98")
gas_e10_df = clean_gas_data("./Online_Data/Gas/95-E10.csv", "E10")
gazole_df = clean_gas_data("./Online_Data/Gas/gazole.csv", "gazole")

merged_rer['Période'] = pd.to_datetime(merged_rer['nearest_datetime']).dt.strftime('%Y-%m')
merged_rer = merged_rer.merge(gas_95_df, on='Période', how='left')
merged_rer = merged_rer.merge(gas_98_df, on='Période', how='left')
merged_rer = merged_rer.merge(gas_e10_df, on='Période', how='left')
merged_rer = merged_rer.merge(gazole_df, on='Période', how='left')

merged_metro.drop(columns=['timestamp'], inplace=True)
#merged_rer.drop(columns=['Période', 'timestamp'], inplace=True)
merged_rer = merged_rer[~merged_rer.duplicated(subset=merged_rer.columns.difference(['delayed_status']).tolist(), keep='last')]

merged_rer.loc[merged_rer['scheduled_arrival'].dt.month == 1, 'E10'] = 1.76
merged_rer.loc[merged_rer['scheduled_arrival'].dt.month == 1, '95'] = 1.79
merged_rer.loc[merged_rer['scheduled_arrival'].dt.month == 1, '98'] = 1.87
merged_rer.loc[merged_rer['scheduled_arrival'].dt.month == 1, 'gazole'] = 1.70

In [15]:
#Weather Variable

#hourly_weather['date'] = pd.to_datetime(hourly_weather['date']) - don't need hourly?
minutely_15_weather['date'] = pd.to_datetime(minutely_15_weather['date'])

merged_metro = pd.merge_asof(merged_metro, 
                   minutely_15_weather, 
                   left_on='real_arrival', 
                   right_on='date', 
                   direction='nearest')

merged_rer = pd.merge_asof(merged_rer, 
                   minutely_15_weather, 
                   left_on='nearest_datetime', 
                   right_on='date', 
                   direction='nearest')

In [16]:
#Add holidays
merged_rer['day'] = pd.to_datetime(merged_rer['nearest_datetime']).dt.normalize()
merged_rer = merged_rer.merge(df_with_holidays, left_on='day', right_on='Date', how='left')
merged_rer['hour'] = merged_rer['nearest_datetime'].dt.hour.astype('int64')

merged_metro['day'] = pd.to_datetime(merged_metro['real_arrival']).dt.normalize()
merged_metro = merged_metro.merge(df_with_holidays, left_on='day', right_on='Date', how='left')
merged_metro['hour'] = merged_metro['real_arrival'].dt.hour.astype('int64')

merged_metro = merged_metro.sort_values(by='real_arrival')
merged_rer = merged_rer.sort_values(by='nearest_datetime')

In [17]:
#Split by stop
CDG = merged_metro[merged_metro['stop_name'].isin(['Charles de Gaulle-Etoile'])]
SGP = merged_metro[merged_metro['stop_name'].isin(['Saint-Germain des Prés'])]
BL = merged_metro[merged_metro['stop_name'].isin(['Blanche'])]
SL = merged_metro[merged_metro['stop_name'].isin(['Saint-Lazare'])]
APK = merged_rer[merged_rer['stop_name'].isin(['Avenue du Président Kennedy'])]
CLH = merged_rer[merged_rer['stop_name'].isin(['Châtelet - Les Halles'])]
GBG = merged_rer[merged_rer['stop_name'].isin(['Gare de Buno Gironville'])]
MP = merged_rer[merged_rer['stop_name'].isin(['Massy - Palaiseau'])]

CDG = CDG.drop_duplicates(subset=['destination_name', 'real_arrival'], keep='first')
SGP = SGP.drop_duplicates(subset=['destination_name', 'real_arrival'], keep='first')
BL = BL.drop_duplicates(subset=['destination_name', 'real_arrival'], keep='first')
SL = SL.drop_duplicates(subset=['destination_name', 'real_arrival'], keep='first')

APK = APK.drop_duplicates(subset=['destination_name', 'scheduled_arrival'], keep='first')
CLH = CLH.drop_duplicates(subset=['destination_name', 'scheduled_arrival'], keep='first')
GBG = GBG.drop_duplicates(subset=['destination_name', 'scheduled_arrival'], keep='first')
MP = MP.drop_duplicates(subset=['destination_name', 'scheduled_arrival'], keep='first')

In [18]:
CDG_traffic = trafic2023_ratio_metro[trafic2023_ratio_metro['LIBELLE_ARRET'] == 'CH.D.G.ETOILE']
SGP_traffic = trafic2023_ratio_metro[trafic2023_ratio_metro['LIBELLE_ARRET'] == 'ST-GERM.D.PRES']
BL_traffic = trafic2023_ratio_metro[trafic2023_ratio_metro['LIBELLE_ARRET'] == 'BLANCHE']
SL_traffic = trafic2023_ratio_metro[trafic2023_ratio_metro['LIBELLE_ARRET'] == 'SAINT-LAZARE']
APK_traffic = trafic2023_ratio_rer[trafic2023_ratio_rer['LIBELLE_ARRET'] == 'AVENUE DU PRESIDENT KENNEDY']
CLH_traffic = trafic2023_ratio_rer[trafic2023_ratio_rer['LIBELLE_ARRET'] == 'CHATELET']
GBG_traffic = trafic2023_ratio_rer[trafic2023_ratio_rer['LIBELLE_ARRET'] == 'BUNO GIRONVILLE']
MP_traffic = trafic2023_ratio_rer[trafic2023_ratio_rer['LIBELLE_ARRET'] == 'MASSY PALAISEAU']

CDG = CDG.merge(CDG_traffic, left_on=['day_type', 'hour'], right_on=['CAT_JOUR', 'start_hour'], how='left')
SGP = SGP.merge(SGP_traffic, left_on=['day_type', 'hour'], right_on=['CAT_JOUR', 'start_hour'], how='left')
BL = BL.merge(BL_traffic, left_on=['day_type', 'hour'], right_on=['CAT_JOUR', 'start_hour'], how='left')
SL = SL.merge(SL_traffic, left_on=['day_type', 'hour'], right_on=['CAT_JOUR', 'start_hour'], how='left')
APK = APK.merge(APK_traffic, left_on=['day_type', 'hour'], right_on=['CAT_JOUR', 'start_hour'], how='left')
CLH = CLH.merge(CLH_traffic, left_on=['day_type', 'hour'], right_on=['CAT_JOUR', 'start_hour'], how='left')
GBG = GBG.merge(GBG_traffic, left_on=['day_type', 'hour'], right_on=['CAT_JOUR', 'start_hour'], how='left')
MP = MP.merge(MP_traffic, left_on=['day_type', 'hour'], right_on=['CAT_JOUR', 'start_hour'], how='left')

In [19]:
#Drop uneccesary columns
dfs = [CDG, SGP, BL, SL, APK, CLH, GBG, MP]

for df in dfs:
    df.drop(columns=['stop_reference', 'line_ref', 'day', 'date', 'Nom de la période', 'nom_jour_ferie', 'holiday_type', 'hour', 'CODE_STIF_TRNS', 'LIBELLE_ARRET', 'CAT_JOUR', 'TRNC_HORR_60', 'start_hour', 'end_hour'], inplace=True)


In [20]:
#To csv
for df, name in zip(dfs, ['CDG', 'SGP', 'BL', 'SL', 'APK', 'CLH', 'GBG', 'MP']):
    df.to_csv(f'./Stations/{name}.csv', index=False)

In [21]:
CLH.shape

(42906, 35)

In [22]:
#rer_incident['timestamp'] = pd.to_datetime(rer_incident['timestamp'])

# travaux dummy?